In [1]:
import torchdrug

In [2]:
torchdrug.__version__

'0.2.0'

In [3]:
from torchdrug import datasets

BASE_PATH = "/home/ec2-user/esm/"


In [4]:
from torchdrug import transforms
from torchdrug import datasets

truncate_transform = transforms.TruncateProtein(max_length=1024, random=False)
protein_view_transform = transforms.ProteinView(view="residue")
transform = transforms.Compose([truncate_transform, protein_view_transform])
# dataset = datasets.SubcellularLocalization(SUBCELLULAR_PATH, atom_feature=None, bond_feature=None, residue_feature="default", transform=transform)
dataset = datasets.BetaLactamase(BASE_PATH, atom_feature=None, bond_feature=None, residue_feature="default", transform=transform)

00:34:05   Downloading https://miladeepgraphlearningproteindata.s3.us-east-2.amazonaws.com/peerdata/beta_lactamase.tar.gz to /home/ec2-user/esm/beta_lactamase.tar.gz
00:34:05   Extracting /home/ec2-user/esm/beta_lactamase.tar.gz to /home/ec2-user/esm


Constructing proteins from sequences: 100%|██████████| 5198/5198 [00:07<00:00, 717.99it/s]


In [5]:
dataset[0]

{'graph': Protein(num_atom=0, num_bond=0, num_residue=286),
 'scaled_effect1': 0.9426838159561157}

In [6]:
train_set, valid_set, test_set = dataset.split()

In [7]:
print("The label of first sample: ", dataset[0][dataset.target_fields[0]])
print("train samples: %d, valid samples: %d, test samples: %d" % (len(train_set), len(valid_set), len(test_set)))

The label of first sample:  0.9426838159561157
train samples: 4158, valid samples: 520, test samples: 520


In [8]:
prot_seq = dataset[0]['graph'].to_sequence().replace('.', '')

In [9]:
import pandas as pd
from tqdm import tqdm

seq = []
for item in tqdm(train_set):
    aa = item['graph'].to_sequence().replace('.', '')
    lf = item['scaled_effect1']
    seq.append({'seq': aa, 'loc': lf, 'split': 'train'})

for item in tqdm(valid_set):
    aa = item['graph'].to_sequence().replace('.', '')
    lf = item['scaled_effect1']
    seq.append({'seq': aa, 'loc': lf, 'split': 'val'})

for item in tqdm(test_set):
    aa = item['graph'].to_sequence().replace('.', '')
    lf = item['scaled_effect1']
    seq.append({'seq': aa, 'loc': lf, 'split': 'test'})

seq = pd.DataFrame(seq)

100%|██████████| 520/520 [00:00<00:00, 1920.49it/s]


In [10]:
seq.to_csv('protein_scaled_effect1.csv')

## Train flourescence model

In [11]:
import pandas as pd

seq = pd.read_csv('protein_scaled_effect1.csv', index_col=0)

In [12]:
seq['seq'].iloc[0]

'MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLILTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPAAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW'

In [13]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

conversion = 'ARNDCQEGHILKMFPSTWYVX'
amino_acids = np.array([a for a in conversion])
aa = seq['seq'].iloc[0]
onehot_encoder = OneHotEncoder(sparse=False, categories=[amino_acids])


In [14]:
sequence = seq['seq'].iloc[0]
sequence_array = np.array(list(sequence)).reshape(-1, 1)
onehot_encoded = onehot_encoder.fit_transform(sequence_array)

In [16]:
from tqdm import tqdm
import numpy as np

embed = np.zeros((len(seq), 286, 21))

for i, sequence in tqdm(enumerate(seq['seq'])):
    sequence_array = np.array(list(sequence)).reshape(-1, 1)
    onehot_encoded = onehot_encoder.fit_transform(sequence_array)
    embed[i, :onehot_encoded.shape[0]] = onehot_encoded

5198it [00:04, 1145.16it/s]


In [17]:
embed = embed.reshape(len(seq), -1)

In [18]:
embed.shape

(5198, 6006)

In [19]:
index = (seq['split'] == 'train').values
X_train = embed[index]
y_train = seq[index]['loc']

index = (seq['split'] == 'val').values
X_val = embed[index]
y_val = seq[index]['loc']

index = (seq['split'] == 'test').values
X_test = embed[index]
y_test = seq[index]['loc']

In [20]:
X_train.shape

(4158, 6006)

In [21]:
from sklearn.linear_model import Ridge

alpha = 0.5
clf = Ridge(alpha=alpha)
clf.fit(X_train, y_train)

Ridge(alpha=0.5)

In [22]:
y_pred = clf.predict(X_test)

In [23]:
from scipy.stats import spearmanr

spearmanr(y_test, y_pred)

SpearmanrResult(correlation=0.7245350761323257, pvalue=9.098781282748791e-86)

In [25]:
# Extract the learned parameters
weights = clf.coef_
intercept = clf.intercept_

In [26]:
# Save the parameters
np.save('weights_beta.npy', weights)
np.save('intercept_beta.npy', intercept)

In [27]:
import torch

# Load the parameters
weights = np.load('weights_beta.npy')
intercept = np.load('intercept_beta.npy')

# Convert to PyTorch tensors
weights_torch = torch.from_numpy(weights)
intercept_torch = torch.from_numpy(np.array([intercept]))

# Define a linear layer
linear_layer = torch.nn.Linear(weights.shape[0], 1)

# Set the weights and bias
with torch.no_grad():  # We don't want these operations to be tracked by the autograd
    linear_layer.weight.data = weights_torch
    linear_layer.bias.data = intercept_torch

In [28]:
linear_layer

Linear(in_features=6006, out_features=1, bias=True)

## Define potential

In [29]:
import os, sys
from sklearn.preprocessing import OneHotEncoder
import numpy as np



# TEMPLATE CLASS
class Potential:
    
    def get_gradients(seq):
        '''
            EVERY POTENTIAL CLASS MUST RETURN GRADIENTS
        '''
        
        sys.exit('ERROR POTENTIAL HAS NOT BEEN IMPLEMENTED')

In [ ]:
class GFP_beta_lactam(Potential):
    """
    Potential for beta lactam
    """    
    def __init__(self, args, features, potential_scale, DEVICE):
        weights = args['weights']
        intercept = args['intercept']
        
        # Convert to PyTorch tensors
        weights_torch = torch.from_numpy(weights)
        intercept_torch = torch.from_numpy(np.array([intercept]))

        # Define a linear layer
        linear_layer = torch.nn.Linear(weights.shape[0], 1)

        # Set the weights and bias
        with torch.no_grad():  # We don't want these operations to be tracked by the autograd
            linear_layer.weight.data = weights_torch
            linear_layer.bias.data = intercept_torch        
        
        self.linear_layer = linear_layer.to(DEVICE)
        self.potential_scale = potential_scale
        self.sequence_length = 286
        
    def get_gradients(self, seq):
        """
        Calculate gradients with respect to activity

        Arguments
        ---------
        seq : tensor
            L X 21 logits after saving seq_out from xt

        Returns
        -------
        gradients : list of tensors
            gradients of seq with respect to flourescence
        """
        soft_seq = torch.softmax(seq, dim=1)

        if soft_seq.shape[0] > self.sequence_length:
            soft_seq = soft_seq[:self.sequence_length]

        if soft_seq.shape[0] < self.sequence_length:
            zeros = torch.zeros(self.sequence_length - soft_seq.shape[0], soft_seq.shape[1])
            soft_seq = torch.cat((soft_seq, zeros), 0)

        score = linear_layer(soft_seq.reshape(-1))
        score.backward()
        gradients = soft_seq.grad

        return gradients * self.potential_scale